# 之前的版本

# 阳光算法

In [13]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


def multi_threshold_sunlight_algorithm(image):
    # Resize for faster processing
    image = cv2.resize(image, (320, 240))
    
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Step 1: First Otsu threshold to get bright (sunlight) vs others
    _, th1 = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Invert to isolate bright regions (likely sunlight)
    sunlight_mask = cv2.bitwise_not(th1)

    # Step 2: Second Otsu on the remaining part (likely track and background)
    non_sunlight = cv2.bitwise_and(gray, gray, mask=sunlight_mask)
    _, th2 = cv2.threshold(non_sunlight, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Get track region
    track_mask = cv2.bitwise_and(non_sunlight, non_sunlight, mask=th2)

    # Step 3: Extract blue region using HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_blue = np.array([90, 50, 50])
    upper_blue = np.array([140, 255, 255])
    blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Combine masks into a label map (just for visualization)
    result = np.zeros_like(image)
    result[sunlight_mask == 0] = [255, 255, 255]      # Sunlight (white)
    result[th2 > 0] = [128, 128, 128]                 # Track (gray)
    result[blue_mask > 0] = [255, 0, 0]               # Blue edge (blue)

    return result

def process_frame(frame):
    """
    处理帧：将帧转换为灰度图并应用大津法二值化。
    """
    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # 转换为灰度图
    # _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)  # 大津法二值化
    # frame = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)  # 转回 BGR 格式
    frame = multi_threshold_sunlight_algorithm(frame)
    return frame

# 输入文件路径
file_name = "exp1"
input_file = f'./data/{file_name}.mp4'
output_file = f'./result/{file_name}_pro.mp4'

# 打开视频文件
cap = cv2.VideoCapture(input_file)

# 获取视频属性
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 使用 MP4 编码

# 创建 VideoWriter 对象
out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width * 2, frame_height))  # 宽度加倍

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # 处理帧
    processed_frame = process_frame(frame)
    
    # 将原始帧和处理后的帧拼接在一起
    combined_frame = cv2.hconcat([frame, processed_frame])
    
    # 写入拼接后的帧
    out.write(combined_frame)

# 释放资源
cap.release()
out.release()
cv2.destroyAllWindows()

# 高斯自适应阈值

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def filter(binary):
    binary = cv2.dilate(binary, np.ones((3, 3), np.uint8), iterations=1)  # 膨胀操作
    binary = cv2.erode(binary, np.ones((3, 3), np.uint8), iterations=1)   # 腐蚀操作
    return binary

def get_the_area(binary):
    # 分水岭算法
    
    
    

def process_frame(frame):
    image = frame   
    image = cv2.resize(image, (320, 240))  # 缩小图像以加快处理速度
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # 转换为灰度图
    binary = cv2.adaptiveThreshold(
        image,
        maxValue=255,
        adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        thresholdType=cv2.THRESH_BINARY,
        blockSize=35,     # 邻域大小（建议奇数）
        C=5               # 调整偏移值
    )
    
    # 形态学滤波
    binary = filter(binary)
    # 搜索连通域
    binary = get_the_area(binary)
    result = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)  # 转回 BGR 格式
    frame = result
    return frame

# 输入文件路径
file_name = "exp1"
input_file = f'./data/{file_name}.mp4'
output_file = f'./result/{file_name}_pro2.mp4'

# 打开视频文件
cap = cv2.VideoCapture(input_file)

# 获取视频属性
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 使用 MP4 编码

# 创建 VideoWriter 对象
out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width * 2, frame_height))  # 宽度加倍

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # 处理帧
    processed_frame = process_frame(frame)
    
    # 将原始帧和处理后的帧拼接在一起
    combined_frame = cv2.hconcat([frame, processed_frame])
    
    # 写入拼接后的帧
    out.write(combined_frame)

# 释放资源
cap.release()
out.release()
cv2.destroyAllWindows()

# 背景估计 + 全局阈值

In [15]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def process_frame(frame):
    img = frame
    # 方法 A：高斯模糊方式估算背景
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 转换为灰度图
    # bg = cv2.GaussianBlur(img, (51, 51), 0)

    # 方法 B：形态学开运算估计背景
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (51, 51))
    bg = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)

    # 计算校正图像
    corrected = cv2.subtract(img, bg)
    corrected = cv2.normalize(corrected, None, 0, 255, cv2.NORM_MINMAX)

    # 可选：直方图均衡化增强对比度
    equ = cv2.equalizeHist(corrected)

    # 全局阈值（二值化）
    _, binary = cv2.threshold(equ, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    result = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)  # 转回 BGR 格式
    # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # 转换为灰度图
    # _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)  # 大津法二值化
    # frame = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)  # 转回 BGR 格式
    frame = result
    return frame

# 输入文件路径
file_name = "exp3"
input_file = f'./data/{file_name}.mp4'
output_file = f'./result/{file_name}_pro.mp4'

# 打开视频文件
cap = cv2.VideoCapture(input_file)

# 获取视频属性
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 使用 MP4 编码

# 创建 VideoWriter 对象
out = cv2.VideoWriter(output_file, fourcc, fps, (frame_width * 2, frame_height))  # 宽度加倍

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # 处理帧
    processed_frame = process_frame(frame)
    
    # 将原始帧和处理后的帧拼接在一起
    combined_frame = cv2.hconcat([frame, processed_frame])
    
    # 写入拼接后的帧
    out.write(combined_frame)

# 释放资源
cap.release()
out.release()
cv2.destroyAllWindows()